In [4]:
import pandas as pd
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
 
from bs4 import BeautifulSoup
import requests

In [2]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [3]:
token = tokenizer.encode("The best",return_tensors='pt')
int(torch.argmax(model(token).logits)+1)


5

In [34]:
url = "https://www.flipkart.com/realme-gt-master-voyager-grey-256-gb/product-reviews/itmcc74e8302084d?pid=MOBG5JPXRXNKKAYN&lid=LSTMOBG5JPXRXNKKAYNMWUTBT&marketplace=FLIPKART"

result = requests.get(url).text

soup = BeautifulSoup(result, 'html.parser')
pages = int(soup.find('div', class_ = "_2MImiq _1Qnn1K").find('span').text.split(' ')[-1])

dic = {}
for page in range(2,pages+1):
    url = f"https://www.flipkart.com/realme-gt-master-voyager-grey-256-gb/product-reviews/itmcc74e8302084d?pid=MOBG5JPXRXNKKAYN&lid=LSTMOBG5JPXRXNKKAYNMWUTBT&marketplace=FLIPKART&page={page}"
    result = requests.get(url).text
    soup = BeautifulSoup(result, 'html.parser')
    body = soup.find('div', class_ = '_1YokD2 _3Mn1Gg col-9-12')
    try:
        reviews = body.find_all('div', class_ = "t-ZTKy")
    except Exception as e:
        continue
    dic[f'page{page}'] = [review.div.div.text for review in reviews ]

In [38]:
data = []
for i in dic.values():
    for j in i:
        data.append(j)


mobile = pd.DataFrame(np.array(data),columns  = ['Reviews'])
mobile.to_csv('mobile.csv')


In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [6]:
mobile = pd.read_csv('mobile.csv')
mobile.drop(labels = ['Unnamed: 0'], axis = 'columns', inplace = True,)

In [24]:
def sentiment(text: str):
    token = tokenizer.encode(text,return_tensors='pt')
    return int(torch.argmax(model(token).logits)+1)


In [26]:
mobile['Sentiment'] = mobile["Reviews"].apply(sentiment)

In [27]:
mobile.head()

,Reviews,Sentiment
0,"Review after 45 days,Pros:1. Performance is go...",2
1,It's good phone camera is good . battery 🔋 is ...,4
2,Thank you Filpkart.. so fast delivery.. Awasom...,5
3,"Good battery back up, amazing display, very ni...",5
4,Very nice phone ... Super performance... Excel...,4
